Because I am pulling in hmm profiles from many different sources there will be redundancies (e.g. antismash, bigslice, others, all contian PFAM profiles)

The code below contains much of the same code as is used to create the unique ids for the hmm profiles in the socialgene database. I will use it to show how many redundant hmm profiles are present.

In [1]:
from IPython.display import Markdown
from pathlib import Path
import csv
import sys
import os
import re
import hashlib


input_dir = "/home/chase/Documents/socialgene_outdir/hmm_downloads"

model_main = []
iter_int = 0
dict_info = []


def socialgene_id(iter_int):
    return "".join(["sg_hmm_", str(iter_int)])


iterco = 0
for path in Path(input_dir).rglob("*_socialgene"):
    rel_path = path.relative_to(input_dir)
    model_dict = {
        "rel_path": rel_path,
        "models": [],
        "name": [],
        "acc": [],
        "hmm": [None],
    }
    with open(path, "r") as f:
        temp_list = []
        temp_list2 = []
        for line in f:
            iterco += 1
            if line.startswith("ACC "):
                model_dict["acc"] = line.strip().split(maxsplit=1)[1]
            if line.startswith("NAME "):
                model_dict["name"] = line.strip().split(maxsplit=1)[1]
            if line.strip() == "//":
                temp_list.append(line)
                model_dict["models"].append(temp_list)
                model_dict["hmm"].append(temp_list2)
                temp_list = []
                temp_list2 = []
                model_main.append(model_dict)
                model_dict = {
                    "rel_path": rel_path,
                    "models": [],
                    "name": [],
                    "acc": [],
                    "hmm": [None],
                }
            else:
                if line.startswith("HMM ") or len(temp_list2) > 0:
                    temp_list2.append(line)
                temp_list.append(line)

To quickly compare exact profile matches, I took the the entire hmm profile block and saved it in the dict (e.g. model_main[0]["hmm"])
Below I concatenate all the lines within each hmm block and hash the resulting string.

In [2]:
hash_list = [
    hashlib.md5(bytes("".join(i["hmm"][1]), encoding="utf-8")).hexdigest()
    for i in model_main
]

Now we compare the length of all the hmms...

In [3]:
print(len(hash_list))

28050


and the unique list of hmms...

In [5]:
len(set(hash_list))

27506

and the difference between the two... 

In [6]:
len(hash_list) - len(set(hash_list))

544

So, for this iteration there were 514 duplicate hmms (hmm profiles that were completely identical)